## **Mitsuba 3**
This notebook contains experiments done with Mitsuba 3


In [1]:
import mitsuba as mi

In [2]:
mi.variants()

['scalar_rgb',
 'scalar_spectral',
 'scalar_spectral_polarized',
 'llvm_ad_rgb',
 'llvm_ad_mono',
 'llvm_ad_mono_polarized',
 'llvm_ad_spectral',
 'llvm_ad_spectral_polarized',
 'cuda_ad_rgb',
 'cuda_ad_mono',
 'cuda_ad_mono_polarized',
 'cuda_ad_spectral',
 'cuda_ad_spectral_polarized']

In [3]:
mi.set_variant('cuda_ad_rgb')  # Set the desired variant

## **Rendering**

In [9]:
scene = mi.load_file("../Mitsuba 3/scenes/cbox.xml")

In [10]:
image = mi.render(scene, spp=256)

In [12]:
mi.Bitmap(image)

Bitmap[
  pixel_format = rgb,
  component_format = float32,
  size = [256, 256],
  srgb_gamma = 0,
  struct = Struct<12>[
    float32 R; // @0, premultiplied alpha
    float32 G; // @4, premultiplied alpha
    float32 B; // @8, premultiplied alpha
  ],
  data = [ 768 KiB of image data ]
]

In [18]:
cam2 = mi.load_dict({
    "type": "perspective",
    "to_world": mi.ScalarTransform4f.look_at(
        target = [-4.0, 0, 0],
        origin = [3.0, 0.5, 5.0],
        up = [0, 1, 0]
    ),
    'film': {
        'type': 'hdrfilm',
        'width': 360,
        'height': 640
    }
})

In [19]:
image2 = mi.render(scene, sensor=cam2, spp=64)

In [20]:
mi.Bitmap(image2)

Bitmap[
  pixel_format = rgb,
  component_format = float32,
  size = [360, 640],
  srgb_gamma = 0,
  struct = Struct<12>[
    float32 R; // @0, premultiplied alpha
    float32 G; // @4, premultiplied alpha
    float32 B; // @8, premultiplied alpha
  ],
  data = [ 2.64 MiB of image data ]
]

## **Gradient based Optimization**

In [21]:
image_ref = mi.render(scene, spp = 1024)
mi.util.convert_to_bitmap(image_ref)

Bitmap[
  pixel_format = rgb,
  component_format = uint8,
  size = [256, 256],
  srgb_gamma = 1,
  struct = Struct<3>[
    uint8 R; // @0, normalized, gamma, premultiplied alpha
    uint8 G; // @1, normalized, gamma, premultiplied alpha
    uint8 B; // @2, normalized, gamma, premultiplied alpha
  ],
  data = [ 192 KiB of image data ]
]

In [22]:
scene_params = mi.traverse(scene)
type(scene_params)

mitsuba.python.util.SceneParameters

In [23]:
scene_params

SceneParameters[
  --------------------------------------------------------------------------------------------------
  Name                                           Flags    Type              Parent
  --------------------------------------------------------------------------------------------------
  allow_thread_reordering                                 bool              Scene
  sensor.near_clip                                        float             PerspectiveCamera
  sensor.far_clip                                         float             PerspectiveCamera
  sensor.shutter_open                                     float             PerspectiveCamera
  sensor.shutter_open_time                                float             PerspectiveCamera
  sensor.film.size                                        ScalarVector2u    HDRFilm
  sensor.film.crop_size                                   ScalarVector2u    HDRFilm
  sensor.film.crop_offset                                 ScalarPoint2u 

In [25]:
param_key = 'red.reflectance.value'
param_ref = mi.Color3f(scene_params[param_key])
scene_params[param_key] = mi.Color3f(0.01, 0.0, 0.9)

In [27]:
scene_params.update();

In [28]:
image_ref = mi.render(scene, spp = 1024)
mi.util.convert_to_bitmap(image_ref)

Bitmap[
  pixel_format = rgb,
  component_format = uint8,
  size = [256, 256],
  srgb_gamma = 1,
  struct = Struct<3>[
    uint8 R; // @0, normalized, gamma, premultiplied alpha
    uint8 G; // @1, normalized, gamma, premultiplied alpha
    uint8 B; // @2, normalized, gamma, premultiplied alpha
  ],
  data = [ 192 KiB of image data ]
]

In [30]:
opt = mi.ad.Adam(lr=0.05)

In [31]:
opt[param_key] = scene_params[param_key]

In [32]:
scene_params.update(opt);

In [33]:
import drjit as dr 

def mse(image):
    return dr.mean(dr.sqr(image - image_ref))

In [36]:
images = []

for it in range(50):
    image = mi.render(scene, scene_params, spp=8)
    loss = mse(image)
    
    dr.backward(loss)
    opt.step()
    
    opt[param_key] = dr.clamp(opt[param_key], 0.0, 1.0)
    
    scene_params.update(opt)
    
    err_ref = dr.sum(dr.sqr(param_ref - scene_params[param_key]))
    print(f"Iteration {it:02d}: Loss = {loss}, Err_ref = {err_ref[0]:6f}", end='\r')
    images.append(image)
print("\nOptimization Complete")
    

/tmp/ipykernel_26793/222133305.py:4: DeprecationWarning: sqr() is deprecated, please use square(arg)
  return dr.mean(dr.sqr(image - image_ref))
/tmp/ipykernel_26793/770118055.py:10: DeprecationWarning: clamp() is deprecated, please use clip(...)
  opt[param_key] = dr.clamp(opt[param_key], 0.0, 1.0)
/tmp/ipykernel_26793/770118055.py:14: DeprecationWarning: sqr() is deprecated, please use square(arg)
  err_ref = dr.sum(dr.sqr(param_ref - scene_params[param_key]))


Iteration 49: Loss = 0.0104622, Err_ref = 1.043706
Optimization Complete


In [37]:
mi.Bitmap(images[-1])  # show last image

Bitmap[
  pixel_format = rgb,
  component_format = float32,
  size = [256, 256],
  srgb_gamma = 0,
  struct = Struct<12>[
    float32 R; // @0, premultiplied alpha
    float32 G; // @4, premultiplied alpha
    float32 B; // @8, premultiplied alpha
  ],
  data = [ 768 KiB of image data ]
]

In [ ]:

image_ref = mi.render(scene, spp=1024)


# traverse / save original param (optional)
scene_params = mi.traverse(scene)
param_key = 'red.reflectance.value'
param_ref = mi.Color3f(scene_params[param_key])  # ground-truth param

# change scene to initial guess
scene_params[param_key] = mi.Color3f(0.01, 0.0, 0.9)
scene_params.update()

# set up optimizer and AD var
opt = mi.ad.Adam(lr=0.01)          
opt[param_key] = scene_params[param_key]
scene_params.update(opt)

import drjit as dr
def mse(image):
    return dr.mean(dr.sqr(image - image_ref))

images = []
for it in range(200):                
    image = mi.render(scene, scene_params, spp=64)   
    loss = mse(image)

    dr.backward(loss)
    opt.step()

    opt[param_key] = dr.clamp(opt[param_key], 0.0, 1.0)
    scene_params.update(opt)

    err_ref = dr.sum(dr.sqr(param_ref - scene_params[param_key]))
    print(f"Iteration {it:02d}: Loss = {loss}, Err_ref = {err_ref[0]:6f}", end='\r')
    images.append(image)
print("\nOptimization Complete")

# show final
mi.Bitmap(images[-1])

/tmp/ipykernel_26793/4168135234.py:21: DeprecationWarning: sqr() is deprecated, please use square(arg)
  return dr.mean(dr.sqr(image - image_ref))
/tmp/ipykernel_26793/4168135234.py:31: DeprecationWarning: clamp() is deprecated, please use clip(...)
  opt[param_key] = dr.clamp(opt[param_key], 0.0, 1.0)
/tmp/ipykernel_26793/4168135234.py:34: DeprecationWarning: sqr() is deprecated, please use square(arg)
  err_ref = dr.sum(dr.sqr(param_ref - scene_params[param_key]))


Iteration 199: Loss = 0.00139444, Err_ref = 0.000003
Optimization Complete


Bitmap[
  pixel_format = rgb,
  component_format = float32,
  size = [256, 256],
  srgb_gamma = 0,
  struct = Struct<12>[
    float32 R; // @0, premultiplied alpha
    float32 G; // @4, premultiplied alpha
    float32 B; // @8, premultiplied alpha
  ],
  data = [ 768 KiB of image data ]
]